In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,757 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,078 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/comp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
df = spark.read.csv("/content/drive/MyDrive/Programación/PySpark/Titanic/titanic_train.csv", header=True, inferSchema=True)
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [5]:
# Contar el número de valores nulos en cada columna
null_counts = [(col_name, df.where(F.col(col_name).isNull()).count()) for col_name in df.columns]

# Imprimir el resultado
for col_name, count in null_counts:
    print(f"Columna '{col_name}': {count} valores nulos")

Columna 'PassengerId': 0 valores nulos
Columna 'Survived': 0 valores nulos
Columna 'Pclass': 0 valores nulos
Columna 'Name': 0 valores nulos
Columna 'Sex': 0 valores nulos
Columna 'Age': 177 valores nulos
Columna 'SibSp': 0 valores nulos
Columna 'Parch': 0 valores nulos
Columna 'Ticket': 0 valores nulos
Columna 'Fare': 0 valores nulos
Columna 'Cabin': 687 valores nulos
Columna 'Embarked': 2 valores nulos


In [6]:
# Elimina la columna 'Cabin' del DataFrame 'df'
df = df.drop('Cabin', 'Name', 'PassengerID', 'Ticket', 'Parch', 'SibSp')

In [7]:
# Elimina las filas con valores nulos solo en las columnas 'Age' y 'Embarked'
df = df.dropna(subset=['Age', 'Embarked'])
df.show()

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|       0|     3|  male|22.0|   7.25|       S|
|       1|     1|female|38.0|71.2833|       C|
|       1|     3|female|26.0|  7.925|       S|
|       1|     1|female|35.0|   53.1|       S|
|       0|     3|  male|35.0|   8.05|       S|
|       0|     1|  male|54.0|51.8625|       S|
|       0|     3|  male| 2.0| 21.075|       S|
|       1|     3|female|27.0|11.1333|       S|
|       1|     2|female|14.0|30.0708|       C|
|       1|     3|female| 4.0|   16.7|       S|
|       1|     1|female|58.0|  26.55|       S|
|       0|     3|  male|20.0|   8.05|       S|
|       0|     3|  male|39.0| 31.275|       S|
|       0|     3|female|14.0| 7.8542|       S|
|       1|     2|female|55.0|   16.0|       S|
|       0|     3|  male| 2.0| 29.125|       Q|
|       0|     3|female|31.0|   18.0|       S|
|       0|     2|  male|35.0|   26.0|       S|
|       1|   

In [8]:
sex_indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
df = sex_indexer.fit(df).transform(df)

# Crear StringIndexer para la columna Embarked
embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
df = embarked_indexer.fit(df).transform(df)

In [9]:
distinct_values = df.select("EmbarkedIndex").distinct()
distinct_values.show()

+-------------+
|EmbarkedIndex|
+-------------+
|          0.0|
|          1.0|
|          2.0|
+-------------+



In [10]:
df.show(5)

+--------+------+------+----+-------+--------+--------+-------------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|SexIndex|EmbarkedIndex|
+--------+------+------+----+-------+--------+--------+-------------+
|       0|     3|  male|22.0|   7.25|       S|     0.0|          0.0|
|       1|     1|female|38.0|71.2833|       C|     1.0|          1.0|
|       1|     3|female|26.0|  7.925|       S|     1.0|          0.0|
|       1|     1|female|35.0|   53.1|       S|     1.0|          0.0|
|       0|     3|  male|35.0|   8.05|       S|     0.0|          0.0|
+--------+------+------+----+-------+--------+--------+-------------+
only showing top 5 rows



In [11]:
df = df.drop('Sex', 'Embarked')

In [12]:
df.show(5)

+--------+------+----+-------+--------+-------------+
|Survived|Pclass| Age|   Fare|SexIndex|EmbarkedIndex|
+--------+------+----+-------+--------+-------------+
|       0|     3|22.0|   7.25|     0.0|          0.0|
|       1|     1|38.0|71.2833|     1.0|          1.0|
|       1|     3|26.0|  7.925|     1.0|          0.0|
|       1|     1|35.0|   53.1|     1.0|          0.0|
|       0|     3|35.0|   8.05|     0.0|          0.0|
+--------+------+----+-------+--------+-------------+
only showing top 5 rows



In [13]:
vectorAssembler = VectorAssembler(inputCols=["Pclass", "Age", "Fare", "SexIndex", "EmbarkedIndex"], outputCol = "features")

In [14]:
scaler = MinMaxScaler(inputCol="features", outputCol="features_scaled")
pipeline = Pipeline(stages=[vectorAssembler, scaler])
scalerModel = pipeline.fit(df)
scaledData = scalerModel.transform(df)
scaledData.show()

+--------+------+----+-------+--------+-------------+--------------------+--------------------+
|Survived|Pclass| Age|   Fare|SexIndex|EmbarkedIndex|            features|     features_scaled|
+--------+------+----+-------+--------+-------------+--------------------+--------------------+
|       0|     3|22.0|   7.25|     0.0|          0.0|[3.0,22.0,7.25,0....|[1.0,0.2711736617...|
|       1|     1|38.0|71.2833|     1.0|          1.0|[1.0,38.0,71.2833...|[0.0,0.4722292033...|
|       1|     3|26.0|  7.925|     1.0|          0.0|[3.0,26.0,7.925,1...|[1.0,0.3214375471...|
|       1|     1|35.0|   53.1|     1.0|          0.0|[1.0,35.0,53.1,1....|[0.0,0.4345312892...|
|       0|     3|35.0|   8.05|     0.0|          0.0|[3.0,35.0,8.05,0....|[1.0,0.4345312892...|
|       0|     1|54.0|51.8625|     0.0|          0.0|[1.0,54.0,51.8625...|(5,[1,2],[0.67328...|
|       0|     3| 2.0| 21.075|     0.0|          0.0|[3.0,2.0,21.075,0...|[1.0,0.0198542347...|
|       1|     3|27.0|11.1333|     1.0| 

In [15]:
train_data, test_data = scaledData.randomSplit([0.8, 0.2], seed=23)

## Decision Tree

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Survived",
featuresCol="features_scaled")
model = dt.fit(train_data)

In [17]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.8


## Perceptron Multicapa

In [18]:
mlpc=MultilayerPerceptronClassifier( featuresCol="features_scaled",labelCol="Survived" ,layers = [5,16,2], maxIter=1000,blockSize=8,seed=7,solver="gd")

In [19]:
ann = mlpc.fit(train_data)

In [20]:
pred = ann.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol='prediction',metricName='accuracy')
accuracy_ann = evaluator.evaluate(pred)
print(f"Test Accuracy: {accuracy_ann}")

Test Accuracy: 0.7931034482758621


## Random Forest

In [21]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier(labelCol="Survived",
featuresCol="features_scaled"
, numTrees=30)
model = rf.fit(train_data)

predictions = model.transform(test_data)

In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.7931034482758621


## Naive Bayes

In [23]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="Survived",
featuresCol="features_scaled")
nbmodel = nb.fit(train_data)

In [24]:
predictions = nbmodel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_nb = evaluator.evaluate(predictions)

print(f"Test Accuracy:{accuracy_nb}")

Test Accuracy:0.7931034482758621


## Regresión Logística

In [25]:
from pyspark.ml.classification import LogisticRegression

logistic_regression = LogisticRegression(featuresCol="features_scaled"
,labelCol="Survived")
model = logistic_regression.fit(train_data)

In [26]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(predictions)

print(f"Test Accuracy:{accuracy_lr}")

Test Accuracy:0.8137931034482758


## Gradiente-boosted Tree

In [27]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Survived"
,featuresCol="features_scaled"
, maxIter=10)
model = gbt.fit(train_data)

In [28]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_gb = evaluator.evaluate(predictions)

print(f"Test Accuracy:{accuracy_gb}")

Test Accuracy:0.8


## Support Vector Machine

In [29]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(labelCol="Survived"
,featuresCol="features_scaled"
,maxIter=10)
lsvcModel = lsvc.fit(train_data)

In [30]:
predictions = lsvcModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_lsvc = evaluator.evaluate(predictions)

print(f"Test Accuracy:{accuracy_lsvc}")

Test Accuracy:0.8
